In [1]:
import os
import subprocess
import re

# --- CONFIGURATION ---
DATATYPE = 'exvivo2'  # Options: 'exvivo', 'exvivo2', 'hcp'
BASEPATH = '/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server/'
THICKNESS_DATA_PATH = f'{BASEPATH}/data/thickness_data/'
# HCP reference surface paths (for standardization)
HCP_SURF_PATHS = {
    'lh': f'{BASEPATH}/HCP_S1200_Atlas_Z4_pkXDZ/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii',
    'rh': f'{BASEPATH}/HCP_S1200_Atlas_Z4_pkXDZ/S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii'
}

def get_base_dir(datatype, basepath):
    if datatype == 'hcp':
        return os.path.join(basepath, 'hcp')
    return os.path.join(basepath, datatype)

def get_boundary_types(datatype):
    if datatype == 'hcp':
        return ['']  # Only total thickness
    return ['inf.pial.', 'wm.inf.', '']

def get_subject_dirs(baseDir):
    return [d for d in os.listdir(baseDir) if os.path.isdir(os.path.join(baseDir, d))]

def smooth_subject_exvivo(baseDir, subject, boundary, radius):
    # Smoothing for exvivo and exvivo2 datasets
    for hemi in ['lh', 'rh']:
        surf_path_check = os.path.join(baseDir, subject, f'{hemi}.inf.32k_fs_LR.surf.gii')
        
        if os.path.exists(surf_path_check):
            surface_path = HCP_SURF_PATHS[hemi]
            shape_in_path = os.path.join(baseDir, subject, f'{hemi}.thickness.{boundary}32k_fs_LR.shape.gii')
            shape_out_path = os.path.join(baseDir, subject, f'{hemi}.thickness.{boundary}32k_{radius}mm_fwhm_fs_LR.shape.gii')
            print(f'Smoothing {subject} {hemi} boundary="{boundary}" radius={radius}')
            print(f'{surface_path} {shape_in_path} {shape_out_path}')
            subprocess.run(
                f"wb_command -metric-smoothing '{surface_path}' '{shape_in_path}' {radius} '{shape_out_path}' -fwhm -fix-zeros",
                shell=True,
                check=True
            )
            print(f'{subject} {hemi} {radius}mm FWHM smoothing complete')
        else:
            print(f'{subject} does not have {boundary} thickness data ({hemi})')
            raise ValueError(f'{subject} does not have {boundary} thickness data ({hemi})')

def smooth_subject_hcp(baseDir, subject, radius):
    # Smoothing for HCP dataset
    for hemi, side in [('L', 'lh'), ('R', 'rh')]:
        shape_in_path = os.path.join(baseDir, subject, f'{subject}.{hemi}.thickness.32k_fs_LR.shape.gii')
        if os.path.exists(shape_in_path):
            surface_path = HCP_SURF_PATHS[side]
            shape_out_path = os.path.join(baseDir, subject, f'{subject}.{hemi}.thickness.32k_{radius}mm_fwhm_fs_LR.shape.gii')
            print(f'Smoothing {subject} HCP {hemi} radius={radius}')
            print(f'{surface_path} {shape_in_path} {shape_out_path}')
            subprocess.run(
                f"wb_command -metric-smoothing '{surface_path}' '{shape_in_path}' {radius} '{shape_out_path}' -fwhm -fix-zeros",
                shell=True,
                check=True
            )
            print(f'{subject} {hemi} {radius}mm FWHM smoothing complete')
        else:
            print(f'{subject} does not have {hemi} thickness data')
            raise ValueError(f'{subject} does not have {hemi} thickness data')

def main():
    baseDir = get_base_dir(DATATYPE, THICKNESS_DATA_PATH)
    boundary_types = get_boundary_types(DATATYPE)
    subject_dirs = get_subject_dirs(baseDir)
    radii = [6]  # FWHM in mm

    for boundary in boundary_types:
        for radius in radii:
            print(f'=== Smoothing with {radius}mm FWHM for boundary="{boundary}" ===')
            for subject in subject_dirs:
                print(f'Processing subject: {subject}')
                try:
                    if re.search(r'exvivo', DATATYPE, re.IGNORECASE):
                        smooth_subject_exvivo(baseDir, subject, boundary, radius)
                    elif DATATYPE == 'hcp':
                        smooth_subject_hcp(baseDir, subject, radius)
                    else:
                        raise ValueError(f'Unknown DATATYPE: {DATATYPE}')
                except Exception as e:
                    print(f'Error with subject {subject}: {e}')
                    continue

if __name__ == "__main__":
    main()


=== Smoothing with 6mm FWHM for boundary="inf.pial." ===
Processing subject: I64
Smoothing I64 lh boundary="inf.pial." radius=6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//HCP_S1200_Atlas_Z4_pkXDZ/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data/exvivo2/I64/lh.thickness.inf.pial.32k_fs_LR.shape.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data/exvivo2/I64/lh.thickness.inf.pial.32k_6mm_fwhm_fs_LR.shape.gii
I64 lh 6mm FWHM smoothing complete
I64 does not have inf.pial. thickness data (rh)
Error with subject I64: I64 does not have inf.pial. thickness data (rh)
Processing subject: I63
I63 does not have inf.pial. thickness data (lh)
Error with subject I63: I63 does not have inf.pial. thickness data (lh)
Processing subject: I55
Smoothing I55 lh boundary="inf.pial." radius=6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-s

In [3]:
#the second code for bigbrain data
import os
import subprocess
radii = [6]  # FWHM in mm
baseDir = f'{THICKNESS_DATA_PATH}/BigBrain/thickness/resample'
#find all the surf.gii files in the baseDir
shape_files = [f for f in os.listdir(baseDir) if f.endswith('.shape.gii')]

for radius in radii:
    for shape_file in shape_files:
        #if the shape_file first characters match lh. 
        if shape_file.startswith('lh.'):
            print(f'{shape_file} {radius}')
            shape_in_path = f'{baseDir}/{shape_file}'
            shape_out_path = f'{baseDir}/{shape_file[:-9]}_{radius}mm_fwhm.shape.gii'
            surface_path = '/Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii'
            print(f'{shape_in_path} {shape_out_path}')
        elif shape_file.startswith('rh.'):
            print(f'{shape_file} {radius}')
            shape_in_path = f'{baseDir}/{shape_file}'
            shape_out_path = f'{baseDir}/{shape_file[:-9]}_{radius}mm_fwhm.shape.gii'
            surface_path = '/Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii'
            print(f'{shape_in_path} {shape_out_path}')
        else:
            #throw error
            raise ValueError(f'Shape file {shape_file} does not start with lh. or rh.')
        subprocess.run(f"""
                        wb_command -metric-smoothing '{surface_path}' '{shape_in_path}' {radius} '{shape_out_path}' -fwhm -fix-zeros
                        """,shell=True)


              
#wb_command -metric-smoothing '{surface_path}' '{shape_in_path}' {radius} '{shape_out_path}' -fwhm -fix-zeros


lh.0-4.32k.shape.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.0-4.32k.shape.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.0-4.32k._6mm_fwhm.shape.gii
rh.5-6.32k._6mm_fwhm._6mm_fwhm._6mm_fwhm._6mm_fwhm.shape.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.5-6.32k._6mm_fwhm._6mm_fwhm._6mm_fwhm._6mm_fwhm.shape.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.5-6.32k._6mm_fwhm._6mm_fwhm._6mm_fwhm._6mm_fwhm._6mm_fwhm.shape.gii
rh.3-4.164k.shape.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.3-4.164k.shape.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data


While running:
/Applications/workbench/bin_macosxub/../macosxub_apps/wb_command.app/Contents/MacOS/wb_command -metric-smoothing /Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.3-4.164k.shape.gii 6 /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.3-4.164k._6mm_fwhm.shape.gii -fwhm -fix-zeros

ERROR: metric does not match surface in number of vertices



rh.0-1.32k._6mm_fwhm._6mm_fwhm._6mm_fwhm._6mm_fwhm.shape.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.0-1.32k._6mm_fwhm._6mm_fwhm._6mm_fwhm._6mm_fwhm.shape.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.0-1.32k._6mm_fwhm._6mm_fwhm._6mm_fwhm._6mm_fwhm._6mm_fwhm.shape.gii
rh.0-6.32k._6mm_fwhm._6mm_fwhm.shape.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.0-6.32k._6mm_fwhm._6mm_fwhm.shape.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.0-6.32k._6mm_fwhm._6mm_fwhm._6mm_fwhm.shape.gii
rh.3-4.32k.shape.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.3-4.32k.shape.gii /Users/dennis.jungchildmind.org


While running:
/Applications/workbench/bin_macosxub/../macosxub_apps/wb_command.app/Contents/MacOS/wb_command -metric-smoothing /Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.3-6.164k.shape.gii 6 /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.3-6.164k._6mm_fwhm.shape.gii -fwhm -fix-zeros

ERROR: metric does not match surface in number of vertices



lh.3-6.32k._6mm_fwhm._6mm_fwhm._6mm_fwhm.shape.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.3-6.32k._6mm_fwhm._6mm_fwhm._6mm_fwhm.shape.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.3-6.32k._6mm_fwhm._6mm_fwhm._6mm_fwhm._6mm_fwhm.shape.gii
lh.2-6.32k.shape.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.2-6.32k.shape.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.2-6.32k._6mm_fwhm.shape.gii
lh.4-5.32k._6mm_fwhm.shape.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.4-5.32k._6mm_fwhm.shape.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/t

...


While running:
/Applications/workbench/bin_macosxub/../macosxub_apps/wb_command.app/Contents/MacOS/wb_command -metric-smoothing /Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.4-5.164k.shape.gii 6 /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.4-5.164k._6mm_fwhm.shape.gii -fwhm -fix-zeros

ERROR: metric does not match surface in number of vertices


While running:
/Applications/workbench/bin_macosxub/../macosxub_apps/wb_command.app/Contents/MacOS/wb_command -metric-smoothing /Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.0-4.164k.shape.gii 6

rh.3-6.32k.shape.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.3-6.32k.shape.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.3-6.32k._6mm_fwhm.shape.gii
rh.5-6.164k.shape.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.5-6.164k.shape.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.5-6.164k._6mm_fwhm.shape.gii



While running:
/Applications/workbench/bin_macosxub/../macosxub_apps/wb_command.app/Contents/MacOS/wb_command -metric-smoothing /Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.5-6.164k.shape.gii 6 /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.5-6.164k._6mm_fwhm.shape.gii -fwhm -fix-zeros

ERROR: metric does not match surface in number of vertices


While running:
/Applications/workbench/bin_macosxub/../macosxub_apps/wb_command.app/Contents/MacOS/wb_command -metric-smoothing /Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.0-1.164k.shape.gii 6

lh.0-1.164k.shape.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.0-1.164k.shape.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.0-1.164k._6mm_fwhm.shape.gii
lh.0-2.32k.shape.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.0-2.32k.shape.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.0-2.32k._6mm_fwhm.shape.gii
lh.4-5.32k._6mm_fwhm._6mm_fwhm._6mm_fwhm._6mm_fwhm.shape.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.4-5.32k._6mm_fwhm._6mm_fwhm._6mm_fwhm._6mm_fwhm.shape.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.4-5.32


While running:
/Applications/workbench/bin_macosxub/../macosxub_apps/wb_command.app/Contents/MacOS/wb_command -metric-smoothing /Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.R.midthickness_MSMAll.32k_fs_LR.surf.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.0-2.164k.shape.gii 6 /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/rh.0-2.164k._6mm_fwhm.shape.gii -fwhm -fix-zeros

ERROR: metric does not match surface in number of vertices


While running:
/Applications/workbench/bin_macosxub/../macosxub_apps/wb_command.app/Contents/MacOS/wb_command -metric-smoothing /Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//BigBrain/thickness/resample/lh.4-6.164k.shape.gii 6

In [10]:
#last code for macaque ex vivo data
import os
import subprocess
radii = [6]  # FWHM in mm
baseDir = f'{THICKNESS_DATA_PATH}/macaque/'
#find all the surf.gii files in the baseDir
shape_files = [f for f in os.listdir(baseDir) if f.endswith('.func.gii')]
surface_base_path = f'{BASEPATH}/parcellation/Schaefer-macaque/surfaces/'

for radius in radii:
    for shape_file in shape_files:
        #NOTE macaque data is just left hemisphere
        print(f'{shape_file} {radius}')
        shape_in_path = f'{baseDir}/{shape_file}'
        shape_out_path = f'{baseDir}/{shape_file[:-9]}_{radius}mm_fwhm.shape.gii'
        #surface_path = '/Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii'
        surface_path = os.path.join(surface_base_path, 'MacaqueYerkes19.L.midthickness.32k_fs_LR.surf.gii')
        print(f'{shape_in_path} {shape_out_path}')
        
        subprocess.run(f"""
                        wb_command -metric-smoothing '{surface_path}' '{shape_in_path}' {radius} '{shape_out_path}' -fwhm -fix-zeros
                        """,shell=True)


macaque_cortical_thickness_thickness_smooth1.0mm_10k.func.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//macaque//macaque_cortical_thickness_thickness_smooth1.0mm_10k.func.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//macaque//macaque_cortical_thickness_thickness_smooth1.0mm_10k_6mm_fwhm.shape.gii
macaque_supragranular_thickness_thickness_smooth1.0mm_10k.func.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//macaque//macaque_supragranular_thickness_thickness_smooth1.0mm_10k.func.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//macaque//macaque_supragranular_thickness_thickness_smooth1.0mm_10k_6mm_fwhm.shape.gii



While running:
/Applications/workbench/bin_macosxub/../macosxub_apps/wb_command.app/Contents/MacOS/wb_command -metric-smoothing /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//parcellation/Schaefer-macaque/surfaces/MacaqueYerkes19.L.midthickness.32k_fs_LR.surf.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//macaque//macaque_cortical_thickness_thickness_smooth1.0mm_10k.func.gii 6 /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//macaque//macaque_cortical_thickness_thickness_smooth1.0mm_10k_6mm_fwhm.shape.gii -fwhm -fix-zeros

ERROR: metric does not match surface in number of vertices


While running:
/Applications/workbench/bin_macosxub/../macosxub_apps/wb_command.app/Contents/MacOS/wb_command -metric-smoothing /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//parcellation/Schaefer-macaque/surfaces/MacaqueYerkes19.L.midthickness.32k_fs_LR.surf.gii /Users/den

macaque_infragranular_thickness_thickness_smooth1.0mm_10k.func.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//macaque//macaque_infragranular_thickness_thickness_smooth1.0mm_10k.func.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//macaque//macaque_infragranular_thickness_thickness_smooth1.0mm_10k_6mm_fwhm.shape.gii
macaque_supragranular_thickness_thickness_smooth1.0mm_32k.func.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//macaque//macaque_supragranular_thickness_thickness_smooth1.0mm_32k.func.gii /Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//macaque//macaque_supragranular_thickness_thickness_smooth1.0mm_32k_6mm_fwhm.shape.gii
macaque_infragranular_thickness_thickness_smooth1.0mm_32k.func.gii 6
/Users/dennis.jungchildmind.org/Desktop/To-be-uploaded-to-CMI-server//data/thickness_data//macaque//macaq